# COVID-19 Effect on Mortality and Fertility in Orange County, California

### Motivation  
The datafication of the pandemic has allowed the public access to more health-related data at a granular level than ever before. The county that I was assigned, Orange County, California,  had quite a high number of cases given the high population of the county. The lockdowns and mask mandates enforced did prevent people from leaving their homes and the spread of other diseases like the flu, and likely even other kinds of death. However, at the same time, with hospitals being overloaded and sometimes even needing to turn away patients, I wonder how many deaths there were in excess of what is typical to see in Orange County as this may provide evidence that there were actually more COVID-related deaths than is officially being reported, since the healthcare industry may not want to use tests on already deceased persons. Essentially, this question will help answer if we can be certain that there were more deaths due to the pandemic than what the “official” count may suggest, or if the pandemic reduced other causes of death such as from accidents.  

Another angle here is that COVID added a lot of uncertainty to the world. This had many adverse effects on people’s lives, and I am curious to see how this affected fertility. Perhaps during these stressful times, fewer people were confident in the future and their ability to support a child, or perhaps quarantine gave people the opportunity to move forward with their family planning because of the flexibility that working from home provided. I would like to inspect the pandemic and its effects on the lives of people in Orange County, CA from the angle of effect on mortality and fertility.  

### Hypothesis  
My hypothesis related to mortality is that all-cause mortality was higher during the pandemic than pre-pandemic, indicating that there were more deaths due to COVID-19 than were being reported. My hypothesis relating to fertility is that there were fewer births during the pandemic compared to pre-pandemic due to the stress and uncertainty it caused.  

### Data  
- The RAW_us_confirmed_cases.csv file from the [Kaggle repository of John Hopkins University COVID-19 data.](https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university?select=RAW_us_confirmed_cases.csv)  
- The RAW_us_deaths.csv file from the [Kaggle repository of John Hopkins University COVID-19 data.](https://www.kaggle.com/antgoldbloom/covid19-data-from-john-hopkins-university?select=RAW_us_deaths.csv)  
- Deaths from [California Health & Human Services Department](https://data.chhs.ca.gov/dataset/death-profiles-by-county/resource/2e546f88-bba8-4d77-846a-7fb77846cac6)  
- Births from [California Health & Human Services Department](https://data.chhs.ca.gov/dataset/live-birth-profiles-by-county/resource/94a186da-7184-427b-b0b5-224bb0a05cd8)

In [1]:
# Set up file
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Show all columns
pd.set_option('display.max_columns', None)

# Set filepaths
RAW_DATA_PATH = '../data_raw/'
CLEAN_DATA_PATH = '../data_clean/'
RESULTS_PATH = '../results/'

## Import COVID-19 Data from Johns Hopkins  
Import the data, pre-downloaded from the Johns Hopkins Kaggle page, and prepare it to be joined with the CHHS data. This means:  
- Create a datetime index on the 'date' columns
- Filter out unnecessary columns like `Lat`, `Long_`, etc. and only keep the necessary columns for analysis: `Province_State`, `Admin2`, `date`, and `cases` or `covid_deaths`.
- Join these data sets together on the `FIPS` and `date` column.
- Filter on just the data from Orange County, California
- Aggregate the data from daily to monthly

In [2]:
# Import data assuming we are in src folder
cases_raw = pd.read_csv(os.path.join(RAW_DATA_PATH, 'RAW_us_confirmed_cases.csv/RAW_us_confirmed_cases.csv'))
covid_deaths_raw = pd.read_csv(os.path.join(RAW_DATA_PATH, 'RAW_us_deaths.csv'))

# identify Orange County, CA FIPs code
fip = cases_raw.loc[(cases_raw['Province_State'] == 'California') & (cases_raw['Admin2'] == 'Orange'), 'FIPS'].values[0]
print("Orange County FIP: ", str(fip))

# Pivot cases and deaths data to long format
cases = pd.melt(cases_raw, id_vars = cases_raw.columns[0:11], var_name = 'date', value_name = 'cases')
cases['date'] = pd.to_datetime(cases['date'], format = '%m/%d/%y')
cases = cases.loc[:, ['Province_State', 'Admin2', 'FIPS', 'date', 'cases']]
covid_deaths = pd.melt(covid_deaths_raw, id_vars = covid_deaths_raw.columns[0:12], var_name = 'date', value_name = 'covid_deaths')
covid_deaths['date'] = pd.to_datetime(covid_deaths['date'], format = '%m/%d/%y')
# Drop common columns from covid_deaths
covid_deaths = covid_deaths.loc[:, ['FIPS', 'Population', 'date', 'covid_deaths']]

# Print column names
print(cases.columns)
print(covid_deaths.columns)

Orange County FIP:  6059.0
Index(['Province_State', 'Admin2', 'FIPS', 'date', 'cases'], dtype='object')
Index(['FIPS', 'Population', 'date', 'covid_deaths'], dtype='object')


In [3]:
# Combine cases and deaths into one df
covid_df = cases.merge(covid_deaths, how = 'outer', on = ['FIPS', 'date'])
covid_df.to_csv(os.path.join(CLEAN_DATA_PATH, 'us_covid_cases-deaths.csv'), index = False)
covid_df

,Province_State,Admin2,FIPS,date,cases,Population,covid_deaths
0,Alabama,Autauga,1001.0,2020-01-22,0.0,55869,0
1,Alabama,Baldwin,1003.0,2020-01-22,0.0,223234,0
2,Alabama,Barbour,1005.0,2020-01-22,0.0,24686,0
3,Alabama,Bibb,1007.0,2020-01-22,0.0,22394,0
4,Alabama,Blount,1009.0,2020-01-22,0.0,57826,0
...,...,...,...,...,...,...,...
2337649,NaN,NaN,56039.0,2021-12-03,NaN,23464,14
2337650,NaN,NaN,56041.0,2021-12-03,NaN,20226,31
2337651,NaN,NaN,90056.0,2021-12-03,NaN,0,0
2337652,NaN,NaN,56043.0,2021-12-03,NaN,7805,36


In [4]:
# Aggregate data to monthly level instead of daily level to match granularity from CHHS data
covid_df.set_index('date', inplace = True)
covid = covid_df.groupby([pd.Grouper(freq='MS'), 'FIPS', 'Province_State', 'Admin2'])\
    .agg({'cases': np.sum, 'covid_deaths': np.sum, 'Population': np.mean})\
    .reset_index()
# Get just Orange County data
covid = covid.loc[covid['FIPS'] == fip, ]
covid

,date,FIPS,Province_State,Admin2,cases,covid_deaths,Population
217,2020-01-01,6059.0,California,Orange,6.0,0,3175692
3543,2020-02-01,6059.0,California,Orange,29.0,0,3175692
6869,2020-03-01,6059.0,California,Orange,3275.0,24,3175692
10195,2020-04-01,6059.0,California,Orange,48275.0,777,3175692
13521,2020-05-01,6059.0,California,Orange,155575.0,2948,3175692
16847,2020-06-01,6059.0,California,Orange,376841.0,7086,3175692
20173,2020-07-01,6059.0,California,Orange,1057892.0,14501,3175692
23499,2020-08-01,6059.0,California,Orange,1467874.0,25041,3175692
26825,2020-09-01,6059.0,California,Orange,1603209.0,33474,3175692
30151,2020-10-01,6059.0,California,Orange,1850344.0,42748,3175692


## Import data from California's Department of Health & Human Services  

### Deaths
Import the data, extract the necessary types of deaths and strata, and create a date column to prepare it for merging with Johns Hopkins' data:  
- Union the disparate data sets to have one dataset with all data
- Drop unnecessary columns `Geography_Type`, `Strata`, `Strata_Name`, `Data_Extract_Date`, `Annotation_Code`, `Annotation_Desc`, `Cause`
- Keep only `Total Population` strata, and filter to just all cause death, injuries, and suicides. Since COVID data is not stratified, we cannot use the stratified CHHS data.
- Pivot the data to have a separate column for each kind of cause of death

In [5]:
# Get data from CHHS of deaths and drop extract date column
deaths_2021 = pd.read_csv('https://data.chhs.ca.gov/dataset/58619b69-b3cb-41a7-8bfc-fc3a524a9dd4/resource/2e546f88-bba8-4d77-846a-7fb77846cac6/download/2021-11_deaths_provisional_county_month_sup.csv')
deaths_19_20 = pd.read_csv('https://data.chhs.ca.gov/dataset/58619b69-b3cb-41a7-8bfc-fc3a524a9dd4/resource/078185a9-e3a7-403f-a546-b8c582f0a9d8/download/2021-11-29_deaths_final_2019-2020_occurrence_county_month_sup.csv')
deaths_14_18 = pd.read_csv('https://data.chhs.ca.gov/dataset/58619b69-b3cb-41a7-8bfc-fc3a524a9dd4/resource/1a69fe2d-9796-49de-be51-1949d09b242a/download/2021-11-29_deaths_final_2014-2018_occurrence_county_month_sup.csv')
deaths_chhs = pd.concat([deaths_2021, deaths_19_20, deaths_14_18], join = 'outer')

# Drop useless columns
# Filter strata for "Total Population"
# Filter for Orange County
# Filter for All causes (total), Accidents (unintentional injuries), Intentional self-harm (suicide)
deaths_chhs = deaths_chhs.loc[(deaths_chhs['Strata'] == 'Total Population') & 
                              (deaths_chhs['County'] == 'Orange') & 
                              (deaths_chhs['Cause'].isin(['ALL', 'INJ', 'SUI'])), ]\
    .drop(['Geography_Type', 'Strata', 'Strata_Name', 'Data_Extract_Date', 'Annotation_Code', 'Annotation_Desc', 'Cause'], axis = 1)

In [6]:
# Create new date column to join on
deaths_chhs['date'] = pd.to_datetime(deaths_chhs['Year'].astype(str) + '/' + deaths_chhs['Month'].astype(str) + '/01',
                                   format = '%Y/%m/%d')
deaths_chhs.drop(['Year', 'Month'], axis = 1, inplace = True)

# Pivot the data from long to wide
deaths_chhs = deaths_chhs.pivot(index = ['County', 'date'], columns = 'Cause_Desc', values = 'Count').reset_index()
deaths_chhs

Cause_Desc,County,date,Accidents (unintentional injuries),All causes (total),Intentional self-harm (suicide)
0,Orange,2014-01-01,60.0,1798.0,38.0
1,Orange,2014-02-01,62.0,1513.0,19.0
2,Orange,2014-03-01,78.0,1668.0,22.0
3,Orange,2014-04-01,60.0,1589.0,21.0
4,Orange,2014-05-01,58.0,1571.0,23.0
...,...,...,...,...,...
89,Orange,2021-06-01,105.0,1750.0,25.0
90,Orange,2021-07-01,85.0,1864.0,17.0
91,Orange,2021-08-01,58.0,2085.0,16.0
92,Orange,2021-09-01,49.0,2020.0,19.0


### Births  

In [7]:
births_21 = pd.read_csv('https://data.chhs.ca.gov/dataset/bc59b3db-3936-4c9b-9275-c5f4b3dc9023/resource/94a186da-7184-427b-b0b5-224bb0a05cd8/download/2021-11_births_provisional_county_month_sup.csv')
births_60 = pd.read_csv('https://data.chhs.ca.gov/dataset/bc59b3db-3936-4c9b-9275-c5f4b3dc9023/resource/d6c30e46-8618-407a-ba5a-bae308f86a1c/download/2021-10-18_births_final_county_month_sup.csv')
births_chhs = pd.concat([births_21, births_60], join = 'outer')

In [8]:
# Drop useless columns
# Filter strata for "Total Population"
# Filter for Orange County
# Filter for All causes (total), Accidents (unintentional injuries), Intentional self-harm (suicide)
births_chhs = births_chhs.loc[(births_chhs['Strata'] == 'Total Population') & 
                              (births_chhs['County'] == 'Orange') & 
                              (births_chhs['Geography_Type'] == 'Occurrence') & 
                              (births_chhs['Year'] > 2013), ]\
    .drop(['Geography_Type', 'Strata', 'Strata_Name', 'Data_Extract_Date', 'Annotation_Code', 'Annotation_Desc', ], axis = 1)

In [9]:
# Create new date column to join on
births_chhs['date'] = pd.to_datetime(births_chhs['Year'].astype(str) + '/' + births_chhs['Month'].astype(str) + '/01',
                                   format = '%Y/%m/%d')
births_chhs.drop(['Year', 'Month'], axis = 1, inplace = True)
births_chhs.rename({'Count': 'births'}, axis = 1, inplace = True)
births_chhs

,County,births,date
145,Orange,2478.0,2021-01-01
435,Orange,2418.0,2021-02-01
725,Orange,2700.0,2021-03-01
1015,Orange,2665.0,2021-04-01
1305,Orange,2753.0,2021-05-01
...,...,...,...
98861,Orange,2886.0,2020-08-01
99441,Orange,2862.0,2020-09-01
100021,Orange,2784.0,2020-10-01
100601,Orange,2606.0,2020-11-01
